In [2]:
!git clone https://github.com/facebookresearch/VideoPose3D

Cloning into 'VideoPose3D'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 9.53 MiB | 6.33 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
cd VideoPose3D

/content/VideoPose3D


In [4]:
mkdir checkpoint

In [5]:
cd checkpoint

/content/VideoPose3D/checkpoint


In [6]:
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin

--2024-07-18 05:56:20--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.67.181.91, 18.67.181.99, 18.67.181.104, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.67.181.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67892577 (65M) [application/octet-stream]
Saving to: ‘pretrained_h36m_detectron_coco.bin’

pretrained_h36m_det 100%[===================>]  64.75M   242MB/s    in 0.3s    

2024-07-18 05:56:21 (242 MB/s) - ‘pretrained_h36m_detectron_coco.bin’ saved [67892577/67892577]



In [9]:
cd /content

/content


In [11]:
!mkdir inputs
#put all the videos you want to do pose prediction on in this folder

In [12]:
!mkdir output
#this is the npz file with keypoints

In [13]:
!mkdir results
#final video results are stored here

In [14]:
!mkdir coordinates
#coords of keypoints in the video

In [1]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-u3fr4dis
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-u3fr4dis
  Resolved https://github.com/facebookresearch/detectron2.git to commit 2a420edb307c9bdf640f036d3b196bed474b8593
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6173934 sha256=74ed75ac969fe1f5b83761f7d019

In [15]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [16]:
cd /content/VideoPose3D/inference

/content/VideoPose3D/inference


###REPLACE /VideoPose3d/inference/infer_video_d2.py script with the code below

no need to run the code, just replace the script with it.

In [18]:
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor

import subprocess as sp
import numpy as np
import time
import argparse
import sys
import os
import glob

def parse_args():
    parser = argparse.ArgumentParser(description='End-to-end inference')
    parser.add_argument(
        '--cfg',
        dest='cfg',
        help='cfg model file (/path/to/model_config.yaml)',
        default=None,
        type=str
    )
    parser.add_argument(
        '--output-dir',
        dest='output_dir',
        help='directory for visualization pdfs (default: /tmp/infer_simple)',
        default='/tmp/infer_simple',
        type=str
    )
    parser.add_argument(
        '--image-ext',
        dest='image_ext',
        help='image file name extension (default: mp4)',
        default='mp4',
        type=str
    )
    parser.add_argument(
        'im_or_folder', help='image or folder of images', default=None
    )
    if len(sys.argv) == 1:
        parser.print_help()
        sys.exit(1)
    return parser.parse_args()

def get_resolution(filename):
    command = ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
               '-show_entries', 'stream=width,height', '-of', 'csv=p=0', filename]
    pipe = sp.Popen(command, stdout=sp.PIPE, bufsize=-1)
    for line in pipe.stdout:
        w, h = line.decode().strip().split(',')
        return int(w), int(h)

def read_video(filename):
    w, h = get_resolution(filename)

    command = ['ffmpeg',
            '-i', filename,
            '-f', 'image2pipe',
            '-pix_fmt', 'bgr24',
            '-vsync', '0',
            '-vcodec', 'rawvideo', '-']

    pipe = sp.Popen(command, stdout=sp.PIPE, bufsize=-1)
    while True:
        data = pipe.stdout.read(w*h*3)
        if not data:
            break
        yield np.frombuffer(data, dtype='uint8').reshape((h, w, 3))


def main(args):

    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(args.cfg))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(args.cfg)
    predictor = DefaultPredictor(cfg)


    if os.path.isdir(args.im_or_folder):
        im_list = glob.iglob(args.im_or_folder + '/*.' + args.image_ext)
    else:
        im_list = [args.im_or_folder]

    for video_name in im_list:
        out_name = os.path.join(
                args.output_dir, os.path.basename(video_name)
            )
        print('Processing {}'.format(video_name))

        boxes = []
        segments = []
        keypoints = []

        for frame_i, im in enumerate(read_video(video_name)):
            t = time.time()
            outputs = predictor(im)['instances'].to('cpu')

            print('Frame {} processed in {:.3f}s'.format(frame_i, time.time() - t))

            has_bbox = False
            if outputs.has('pred_boxes'):
                bbox_tensor = outputs.pred_boxes.tensor.numpy()
                if len(bbox_tensor) > 0:
                    has_bbox = True
                    scores = outputs.scores.numpy()[:, None]
                    bbox_tensor = np.concatenate((bbox_tensor, scores), axis=1)
            if has_bbox:
                kps = outputs.pred_keypoints.numpy()
                kps_xy = kps[:, :, :2]
                kps_prob = kps[:, :, 2:3]
                kps_logit = np.zeros_like(kps_prob) # Dummy
                kps = np.concatenate((kps_xy, kps_logit, kps_prob), axis=2)
                kps = kps.transpose(0, 2, 1)
            else:
                kps = []
                bbox_tensor = []

            # Mimic Detectron1 format
            cls_boxes = [[], bbox_tensor]
            cls_keyps = [[], kps]

            boxes.append(cls_boxes)
            segments.append(None)
            keypoints.append(cls_keyps)


        # Video resolution
        metadata = {
            'w': im.shape[1],
            'h': im.shape[0],
        }

        datas = {
            'boxes': boxes,
            'segments': segments,
            'keypoints': keypoints,
            'metadata': metadata
        }
        for key, value in datas.items():
            datas[key] = np.array(value, dtype=object)

        np.savez_compressed(out_name, **datas)


if __name__ == '__main__':
    setup_logger()
    args = parse_args()
    main(args)


## After replacing the script, can run all the code cells below

In [20]:
!python infer_video_d2.py \
    --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml \
    --output-dir /content/output/ \
    --image-ext mp4 \
    /content/inputs

[07/18 06:04:21 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x/138363331/model_final_997cc7.pkl ...
Processing /content/inputs/walk1.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse 

In [21]:
cd /content/VideoPose3D/data

/content/VideoPose3D/data


In [22]:
!python prepare_data_2d_custom.py -i /content/output -o myvideos

Parsing 2D detections from /content/output
Processing /content/output/walk1.mp4.npz
283 total frames processed
0 frames were interpolated
----------
Saving...
Done.


In [23]:
cd /content/VideoPose3D

/content/VideoPose3D


In [25]:
#so all the videos are combined into the custom dataset. now we can run pose predictions on one of the videos using the code below
# --viz-subject {name of video you want to run model one}
# --viz-video {path to the video}
# --viz-output {where u want the output video to be stored}
# --viz-export {where you want the coordinates of the keypoints to be stored}
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject walk1.mp4 --viz-action custom --viz-camera 0 --viz-video /content/inputs/walk1.mp4 --viz-output /content/results/walk1_result.mp4 --viz-size 6 --viz-export /content/coordinates/walk1_coordinates.npy

Namespace(dataset='custom', keypoints='myvideos', subjects_train='S1,S5,S6,S7,S8', subjects_test='S9,S11', subjects_unlabeled='', actions='*', checkpoint='checkpoint', checkpoint_frequency=10, resume='', evaluate='pretrained_h36m_detectron_coco.bin', render=True, by_subject=False, export_training_curves=False, stride=1, epochs=60, batch_size=1024, dropout=0.25, learning_rate=0.001, lr_decay=0.95, data_augmentation=True, test_time_augmentation=True, architecture='3,3,3,3,3', causal=False, channels=1024, subset=1, downsample=1, warmup=1, no_eval=False, dense=False, disable_optimizations=False, linear_projection=False, bone_length_term=True, no_proj=False, viz_subject='walk1.mp4', viz_action='custom', viz_camera=0, viz_video='/content/inputs/walk1.mp4', viz_skip=0, viz_output='/content/results/walk1_result.mp4', viz_export='/content/coordinates/walk1_coordinates.npy', viz_bitrate=3000, viz_no_ground_truth=False, viz_limit=-1, viz_downsample=1, viz_size=6)
Loading dataset...
Preparing data

In [29]:
## visualize coordinates
import numpy as np

coordinates = np.load('/content/coordinates/walk1_coordinates.npy')
print(coordinates)
print(coordinates.shape)


[[[ 4.09154745e-06 -2.76590799e-05  3.06225297e-06]
  [-1.27835065e-01  5.95791079e-03 -1.71973705e-02]
  [-7.66793713e-02  3.01770687e-01 -2.25647211e-01]
  ...
  [-1.53635502e-01 -4.05379951e-01 -7.80211836e-02]
  [-2.57844061e-01 -1.72586724e-01  1.17575545e-02]
  [-2.69642055e-01  5.97565770e-02  1.12849548e-02]]

 [[ 4.25412145e-06 -2.77193794e-05  3.06180573e-06]
  [-1.27861947e-01  6.01296499e-03 -1.70075335e-02]
  [-7.78244287e-02  3.03656876e-01 -2.26832896e-01]
  ...
  [-1.54483497e-01 -4.04991806e-01 -7.88244158e-02]
  [-2.58054256e-01 -1.72352389e-01  1.20168366e-02]
  [-2.70157397e-01  6.05728477e-02  1.33675337e-02]]

 [[ 4.33266723e-06 -2.76680857e-05  3.10415840e-06]
  [-1.27869144e-01  5.81290480e-03 -1.69669613e-02]
  [-7.89503306e-02  3.05340350e-01 -2.26818249e-01]
  ...
  [-1.54303327e-01 -4.04935002e-01 -8.04003030e-02]
  [-2.56939441e-01 -1.71614259e-01  1.02679133e-02]
  [-2.69854248e-01  6.18430078e-02  1.16403513e-02]]

 ...

 [[-2.10135113e-06 -2.92398927e-05